In [1]:
import gymnasium as gym

from stable_baselines3 import SAC, PPO

from gymnasium.envs.registration import register, registry
from stable_baselines3.common.evaluation import evaluate_policy
import time

import matplotlib
import matplotlib.pyplot as plt

import torch

2025-01-30 20:36:17.132150: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 20:36:17.145687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738269377.162283    4476 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738269377.166771    4476 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 20:36:17.181383: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='environments:MarineEnv',  # String reference to the class
    )

In [4]:
env_kwargs = dict(
    render_mode='human',
    continuous=True,
    max_episode_steps=2400,
    training_stage=2,
    timescale=1/6
)
env = gym.make('MarineEnv-v0', **env_kwargs)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType' was raised from the environment creator for MarineEnv-v0 with kwargs ({'render_mode': 'human', 'continuous': True, 'training_stage': 2, 'timescale': 0.16666666666666666})

In [ ]:
agent_ppo = PPO(policy='MlpPolicy', env=env, device='cpu').load('ppo_marine_good')

In [ ]:
agent_sac = SAC(policy='MlpPolicy', env=env, device='cuda').load('sac_3')

In [ ]:
timescale = 1 / 6
seed = 42
for _ in range(1):
    env = gym.make('MarineEnv-v0', render_mode='human', continuous=True, training_stage=2, timescale=timescale, training=False, seed=seed)
    state, _ = env.reset()
    print(state)
    episode_rewards = 0 
    for _ in range(int(400 / timescale)):
        action = agent_ppo.predict(state, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action[0])
        env.render()
        time.sleep(0.005)
        episode_rewards += reward
        print('===========================')
        print(observation)
        print(reward)
        
        if terminated or truncated:
            print(episode_rewards)
            break
    
        state = observation
            
    print(episode_rewards)
    print(state)
    env.close()